#### Regression Assignment
- 1. Domain -- Supervise Learning
- 2. Type - Regression
- 3. Multiple Linear Regression

Key Testing Categories
1. Baseline Testing (T001)

Default parameters to establish baseline performance

2. Tree Count Variation (T002-T003)

Test impact of different number of trees (50, 100, 200)

3. Criterion Testing (T004-T006)

Test all available split criteria

4. Tree Depth Control (T007-T008)

Test depth limitations for regularization

5. Split Controls (T009-T012)

Test minimum samples required for splits and leaves

6. Feature Selection (T014-T015)

Test different feature subset sizes

7. Leaf Node Limits (T016-T017)

Control model complexity via leaf nodes

8. Bootstrap & OOB (T019-T020)

Test with/without bootstrap and OOB scoring

9. Performance Optimization (T021-T022)

Parallel processing tests

10. Regularization (T025-T026)

Pruning with cost-complexity parameter

11. Subsampling (T027-T028)

Test different sample sizes per tree

12. Combined Parameters (T029-T030)

Complex scenarios with multiple regularization techniques

Notes

Always use random_state=42 for reproducibility in actual tests (except T001)
Monitor training time for each scenario
Track memory usage for large n_estimators
OOB score requires bootstrap=True
n_jobs=-1 uses all available CPU cores
Adjust max_samples based on dataset size


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
import numpy as np


In [10]:
# Define test scenarios
test_scenarios = [
    # T001: Baseline/Default
    {
        'test_id': 'T001',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': None,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Default/Baseline'
    },
    
    # T002: Fewer trees with seed
    {
        'test_id': 'T002',
        'n_estimators': 50,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Fewer trees with seed'
    },
    
    # T003: More trees
    {
        'test_id': 'T003',
        'n_estimators': 200,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'More trees'
    },
    
    # T004: Different criterion (MAE)
    {
        'test_id': 'T004',
        'n_estimators': 100,
        'criterion': 'absolute_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Different criterion (MAE)'
    },
    
    # T005: Friedman MSE criterion
    {
        'test_id': 'T005',
        'n_estimators': 100,
        'criterion': 'friedman_mse',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Friedman MSE criterion'
    },
    
    # T006: Poisson criterion
    {
        'test_id': 'T006',
        'n_estimators': 100,
        'criterion': 'poisson',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Poisson criterion'
    },
    
    # T007: Limited tree depth
    {
        'test_id': 'T007',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': 5,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Limited tree depth'
    },
    
    # T008: Moderate tree depth
    {
        'test_id': 'T008',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': 10,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Moderate tree depth'
    },
    
    # T009: Higher min_samples_split
    {
        'test_id': 'T009',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 5,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Higher min_samples_split'
    },
    
    # T010: Much higher min_samples_split
    {
        'test_id': 'T010',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 10,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Much higher min_samples_split'
    },
    
    # T011: Higher min_samples_leaf
    {
        'test_id': 'T011',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 5,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Higher min_samples_leaf'
    },
    
    # T012: Much higher min_samples_leaf
    {
        'test_id': 'T012',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 10,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Much higher min_samples_leaf'
    },
    
    # T013: With weight fraction leaf
    {
        'test_id': 'T013',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.1,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'With weight fraction leaf'
    },
    
    # T014: Reduced max_features
    {
        'test_id': 'T014',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 0.5,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Reduced max_features'
    },
    
    # T015: Low max_features
    {
        'test_id': 'T015',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 0.3,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Low max_features'
    },
    
    # T016: Limited leaf nodes
    {
        'test_id': 'T016',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': 50,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Limited leaf nodes'
    },
    
    # T017: More leaf nodes
    {
        'test_id': 'T017',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': 100,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'More leaf nodes'
    },
    
    # T018: Min impurity decrease
    {
        'test_id': 'T018',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.01,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Min impurity decrease'
    },
    
    # T019: No bootstrap
    {
        'test_id': 'T019',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': False,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'No bootstrap'
    },
    
    # T020: With OOB score
    {
        'test_id': 'T020',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': True,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'With OOB score'
    },
    
    # T021: Parallel processing (all cores)
    {
        'test_id': 'T021',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': -1,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Parallel processing (all cores)'
    },
    
    # T022: Parallel processing (4 cores)
    {
        'test_id': 'T022',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': 4,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Parallel processing (4 cores)'
    },
    
    # T023: Verbose output
    {
        'test_id': 'T023',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 1,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Verbose output'
    },
    
    # T024: Warm start enabled
    {
        'test_id': 'T024',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': True,
        'ccp_alpha': 0.0,
        'max_samples': None,
        'purpose': 'Warm start enabled'
    },
    
    # T025: Pruning with ccp_alpha
    {
        'test_id': 'T025',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.01,
        'max_samples': None,
        'purpose': 'Pruning with ccp_alpha'
    },
    
    # T026: Higher pruning
    {
        'test_id': 'T026',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.05,
        'max_samples': None,
        'purpose': 'Higher pruning'
    },
    
    # T027: Subsample 80%
    {
        'test_id': 'T027',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': 0.8,
        'purpose': 'Subsample 80%'
    },
    
    # T028: Subsample 50%
    {
        'test_id': 'T028',
        'n_estimators': 100,
        'criterion': 'squared_error',
        'max_depth': None,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': None,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': 0.5,
        'purpose': 'Subsample 50%'
    },
    
    # T029: Combined regularization 1
    {
        'test_id': 'T029',
        'n_estimators': 50,
        'criterion': 'absolute_error',
        'max_depth': 8,
        'min_samples_split': 5,
        'min_samples_leaf': 3,
        'min_weight_fraction_leaf': 0.0,
        'max_features': 0.7,
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'bootstrap': True,
        'oob_score': True,
        'n_jobs': -1,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.0,
        'max_samples': 0.7,
        'purpose': 'Combined regularization 1'
    },
    
    # T030: Combined regularization 2
    {
        'test_id': 'T030',
        'n_estimators': 150,
        'criterion': 'friedman_mse',
        'max_depth': 15,
        'min_samples_split': 10,
        'min_samples_leaf': 5,
        'min_weight_fraction_leaf': 0.05,
        'max_features': 0.5,
        'max_leaf_nodes': 75,
        'min_impurity_decrease': 0.01,
        'bootstrap': True,
        'oob_score': False,
        'n_jobs': 4,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False,
        'ccp_alpha': 0.02,
        'max_samples': 0.6,
        'purpose': 'Combined regularization 2'
    }
]


In [11]:
df = pd.read_csv('insurance_pre.csv')
X = df.drop('charges', axis=1)
y = df['charges']
X = pd.get_dummies(X, drop_first=True)


In [12]:
rscore = []
def run_test_scenario(X_train, X_test, y_train, y_test, scenario):
    test_id = scenario.pop('test_id')
    purpose = scenario.pop('purpose')
    
    #print(f"\nRunning {test_id}: {purpose}")
    print(f"Parameters:  {test_id} : {scenario}")
    
    # Create and train model
    model = RandomForestRegressor(**scenario)
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results = {
        'test_id': test_id,
        'purpose': purpose,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'feature_importances': model.feature_importances_
    }
    
    if scenario.get('oob_score'):
        results['oob_score'] = model.oob_score_
    print(f"Results - r2: R²: {r2:.4f}")

    rscore.append(r2)
    return results


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Run all test scenarios
results = []
for scenario in test_scenarios:
    result = run_test_scenario(X_train, X_test, y_train, y_test, scenario)
    results.append(result)

Parameters:  T001 : {'n_estimators': 100, 'criterion': 'squared_error', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'max_features': 1.0, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'bootstrap': True, 'oob_score': False, 'n_jobs': None, 'random_state': None, 'verbose': 0, 'warm_start': False, 'ccp_alpha': 0.0, 'max_samples': None}
Results - r2: R²: 0.8541
Parameters:  T002 : {'n_estimators': 50, 'criterion': 'squared_error', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'max_features': 1.0, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'bootstrap': True, 'oob_score': False, 'n_jobs': None, 'random_state': 42, 'verbose': 0, 'warm_start': False, 'ccp_alpha': 0.0, 'max_samples': None}
Results - r2: R²: 0.8565
Parameters:  T003 : {'n_estimators': 200, 'criterion': 'squared_error', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


Results - r2: R²: 0.8576
Parameters:  T024 : {'n_estimators': 100, 'criterion': 'squared_error', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'max_features': 1.0, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'bootstrap': True, 'oob_score': False, 'n_jobs': None, 'random_state': 42, 'verbose': 0, 'warm_start': True, 'ccp_alpha': 0.0, 'max_samples': None}
Results - r2: R²: 0.8576
Parameters:  T025 : {'n_estimators': 100, 'criterion': 'squared_error', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'max_features': 1.0, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'bootstrap': True, 'oob_score': False, 'n_jobs': None, 'random_state': 42, 'verbose': 0, 'warm_start': False, 'ccp_alpha': 0.01, 'max_samples': None}
Results - r2: R²: 0.8576
Parameters:  T026 : {'n_estimators': 100, 'criterion': 'squared_error', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 

In [18]:
for n in range(len(rscore)):
    print(f"R² Score for scenario {n}: {rscore[n]:.4f}")
    

R² Score for scenario 0: 0.8541
R² Score for scenario 1: 0.8565
R² Score for scenario 2: 0.8589
R² Score for scenario 3: 0.8608
R² Score for scenario 4: 0.8571
R² Score for scenario 5: 0.8595
R² Score for scenario 6: 0.8713
R² Score for scenario 7: 0.8600
R² Score for scenario 8: 0.8618
R² Score for scenario 9: 0.8656
R² Score for scenario 10: 0.8733
R² Score for scenario 11: 0.8759
R² Score for scenario 12: 0.8504
R² Score for scenario 13: 0.8638
R² Score for scenario 14: 0.8484
R² Score for scenario 15: 0.8652
R² Score for scenario 16: 0.8606
R² Score for scenario 17: 0.8582
R² Score for scenario 18: 0.7142
R² Score for scenario 19: 0.8576
R² Score for scenario 20: 0.8576
R² Score for scenario 21: 0.8576
R² Score for scenario 22: 0.8576
R² Score for scenario 23: 0.8576
R² Score for scenario 24: 0.8576
R² Score for scenario 25: 0.8576
R² Score for scenario 26: 0.8595
R² Score for scenario 27: 0.8696
R² Score for scenario 28: 0.8753
R² Score for scenario 29: 0.7939
